### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some key pros of using AutoGen in your AI Agent project:

1. **Scalability**: AutoGen allows for the creation of modular and extensible systems, making it easy to scale your AI capabilities as your project grows.

2. **Ease of Use**: It integrates observability and debugging tools that simplify the monitoring and control of agent workflows, enhancing overall usability.

3. **High Customization**: AutoGen supports high levels of customization for various applications, allowing you to tailor agents to specific needs and workflows.

4. **Reduced Coordination Complexity**: The framework facilitates natural-language handoffs between agents, minimizing the need for complex inter-agent protocols.

5. **Multi-Agent Collaboration**: AutoGen excels in scenarios requiring collaboration between multiple AI agents, making it ideal for dynamic environments with complex automation needs.

6. **Support for Advanced AI Capabilities**: It is well-suited for applications that require advanced problem-solving and continuous learning, adapting well to diverse use cases.

These advantages position AutoGen as a strong choice for developing AI agents that can operate efficiently and effectively in collaborative environments. 

TERMINATE

## Cons of AutoGen:
Here are some reasons against choosing AutoGen for your AI agent project:

1. **Difficult Documentation**: The documentation is noted to be hard to read and lacks sufficient examples, making it challenging for developers to implement and troubleshoot the framework effectively.

2. **Limited Functionality**: Certain features, such as structured outputs, do not work reliably. This could hinder development and lead to increased frustration among teams trying to integrate those functionalities.

3. **Increased Dependency**: Adding AutoGen as a dependency raises concerns about additional layers of complexity in your project. This could lead to compatibility issues or breakage with future updates to the framework.

4. **Steeper Learning Curve**: Users might experience a steeper learning curve, which could slow down the onboarding of new team members and the overall development process.

5. **Customization Limitations**: AutoGen can be less customizable compared to other alternatives, which might restrict developers' ability to tailor the framework to their specific project needs.

6. **Impact on Project Quality**: The widespread use of AutoGen may inadvertently decrease the quality of outputs if not used judiciously, as it simplifies the content creation process.

7. **Lack of Important Features**: There are concerns about missing features like data encryption and intellectual property control, which are crucial for many applications in today's security-conscious environment.

These points are important to consider as you evaluate whether AutoGen is suitable for your project needs. 

TERMINATE



## Decision:

After evaluating the pros and cons of using AutoGen for the project, I would recommend **against** using AutoGen. 

While it offers scalability, ease of use, and the ability to support multi-agent collaboration, the drawbacks present serious concerns. The difficulties with documentation, limited functionality, and steeper learning curve could significantly hinder development efforts. Additionally, the potential for increased dependency and impact on project quality due to simplification of processes raises red flags. The lack of crucial features like data encryption also poses risks that cannot be overlooked. 

In light of these factors, it may be beneficial to consider alternative solutions that provide a better balance of functionality, support, and security tailored to the project's requirements.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()